In [1]:
import os
import sys
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ipywidgets import interactive

In [2]:
path = "/home/manousos/myfiles/thesis"
startyear = "2014"
endyear = "2014"
loss = "14"

places = {
    1: (35.512, 24.012),
    2: (35.364, 24.471),
    3: (35.343, 25.153),
    4: (35.185, 25.706),
    5: (35.050, 24.877)
}

In [3]:
print("Select pre-defined place from the list below (1-5) "
      +"or press 6 to provide custom coordinates:")

choice = int(input("[1] Chania\n[2] Rethymno\n"
                   +"[3] Heraklio\n[4] Ag.Nikolaos\n"
                   +"[5] Moires\n[6] Custom\n"))

while choice > 6 or choice < 1:
    print("\nInvalid answer. Please choose one of the below:")
    choice = int(input("[1] Chania\n[2] Rethymno\n"
                       +"[3] Heraklio\n[4] Ag.Nikolaos\n"
                       +"[5] Moires\n[6] Custom\n"))


if choice == 6:
    lat = input("Latitude of area: ")
    while float(lat) < -90.0 or float(lat) > 90.0 :
        print("Invalid range for latitude...")
        lat = input("Please provide valid input (-90, 90): ")

    lon = input("Longitude of area: ")
    while float(lon) < -180.0 or float(lon) > 180.0 :
        print("Invalid range for longitude...")
        lat = input("Please provide valid input (-180, 180): ")
else:
    lat = str(places[choice][0])
    lon = str(places[choice][1])


print("\nChoose project duration:")
dur = int(input("[1] Day\n[2] Year\n"))

while dur > 2 or choice < 1:
    print("\nInvalid answer. Please choose one of the below:")
    dur = int(input("[1] Day\n[2] Year\n"))

Select pre-defined place from the list below (1-5) or press 6 to provide custom coordinates:
[1] Chania
[2] Rethymno
[3] Heraklio
[4] Ag.Nikolaos
[5] Moires
[6] Custom
5

Choose project duration:
[1] Day
[2] Year
1


In [4]:
url = ("https://re.jrc.ec.europa.eu/api/seriescalc?lat="
     +lat+"&lon="+lon+"&startyear="+startyear+"&endyear="
     +endyear+"&peakpower=1&angle="+lat
     +"&loss="+loss+"&pvcalculation=1")
r = requests.get(url)

if r.status_code == 200:
    print("\nConnection Established!\nDownloading data...\n")
    os.system("curl \'"+url+"\' | tail -n+11 | head -n-11 >"+path+"/data/pv_formatted.csv")
    print("\nSaved data to file pv_formatted.csv")

else:
    sys.exit("\nConnection failed\nStatus Code: " + str(r.status_code))
    

gridload = pd.read_csv(path + "/data/moires_gridload.csv", sep=":")
pv = pd.read_csv(path + "/data/pv_formatted.csv")


Connection Established!


Saved data to file pv_formatted.csv


In [5]:
def year(solar):
    global y_grid, y_pv, period
    
    y_grid = gridload.drop(gridload.columns[0:2], axis=1).stack().array
    y_pv = pv['P'] * solar / 1_000_000 # converting to MW from W

    x = range(8760)
    xlabel = 'Hour of the day'
    period = 'year'
    
    plt.plot(x, y_grid)
    plt.plot(x, y_pv)

    plt.xlabel(xlabel)
    plt.ylabel('Power (MW)')
    plt.title('Grid load and Power output of PV')

    plt.grid(True)
    plt.legend(['Grid Load', 'PV output'])
    plt.show()


def day(solar):
    global y_grid, y_pv, period
    
    y_grid = gridload.drop(gridload.columns[0:2], axis=1).mean()
    
    pv_conversion = pv['P'] * solar
    data_pv = pv_conversion.iloc[::24] / 1_000_000 # converting to MW from W
    data_pv = data_pv.to_frame().rename(columns={'P': 0}).reset_index().drop(columns="index")

    for i in range(1,24):
        filter = pv_conversion.iloc[i::24] / 1_000_000
        df_newcol = pd.DataFrame(filter)
        df_newcol = df_newcol.reset_index().drop(columns="index")
        data_pv[i] = df_newcol
    
    y_pv = data_pv.mean()

    x = range(24)
    xlabel = 'Hour of the day'
    period = 'day'
    
    plt.plot(x, y_grid)
    plt.plot(x, y_pv)

    plt.xlabel(xlabel)
    plt.ylabel('Power (MW)')
    plt.title('Grid load and Power output of PV')

    plt.grid(True)
    plt.legend(['Grid Load', 'PV output'])
    plt.show()

In [6]:
if dur == 1:
    interactive_plot = interactive(day, solar=(1000, 30000, 1000))
else:
    interactive_plot = interactive(year, solar=(1000, 30000, 1000))

interactive_plot

interactive(children=(IntSlider(value=15000, description='solar', max=30000, min=1000, step=1000), Output()), …

In [9]:
balance = np.array(y_pv.array - y_grid.array)
excess_power = balance[balance > 0].sum()
excess_power_duration = balance[balance > 0].size
excess_energy = excess_power * excess_power_duration

print(f'\n{excess_energy:.5} MWh are wasted each ' + period + '!')


0.0 MWh are wasted each day!


In [8]:
with open(path+"/data/battery.json", 'r') as f:
    data = json.load(f)
    bat = pd.DataFrame(data, index=[0])

capacity = bat['capacity'].values / 1000 # converting to MWh from kWh
batteries_needed = excess_energy // capacity

print(f'{int(batteries_needed[0])} batteries are needed to '
        +'store this excess energy.')

0 batteries are needed to store this excess energy.
